In [1]:
#conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1

In [2]:
#pip install tensorflow==2.11

In [3]:
#conda install -c conda-forge cudatoolkit=11.2

In [4]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import tensorflow.keras as keras

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.11.0
Keras Version: 2.11.0

Python 3.9.16 (main, Mar  8 2023, 14:00:05) 
[GCC 11.2.0]
Pandas 1.5.3
Scikit-Learn 1.2.2
GPU is available


In [2]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_predict, KFold
import sklearn.preprocessing

import time
import sklearn
import numpy as np
import os
import sys
from joblib import parallel_backend

In [3]:
# define a list of datasets
datasets = [
"BEARING_Univar",
"PHM2022_Multivar",
"PHM2022_Univar_PIN",
"PHM2022_Univar_PO",
"PHM2022_Univar_PDIN",
"ETCHING_Multivar",
"MFPT_48KHZ_Univar",
"MFPT_96KHZ_Univar",
"PADERBORN_6KHZ_Univar",
"PADERBORN_4KHZ_Univar",
"PADERBORN_64KHZ_Multivar",
"PADERBORN_4KHZ_Multivar",
"Hydraulic_systems_10HZ_Multivar",
"Hydraulic_systems_100HZ_Multivar",
"Gas_sensors_home_activity",
"Control_charts",
"CWRU_12k_DE_univar",
"CWRU_12k_DE_multivar",
"CWRU_12k_FE_univar",
"CWRU_12k_FE_multivar",
"CWRU_48k_DE_univar",
"CWRU_48k_DE_multivar"
]
#datasets = ["PHM2022_Multivar", "PHM2022_Univar_PDIN"]
datasets_path = "../datasets"

for dataset in datasets:
    Dataset_name = dataset + "_Dataset"
    Dataset = np.load(datasets_path + "/" + Dataset_name + ".npy")
    print(Dataset.shape)
    

    Labels_name = dataset + "_Labels"
    Labels = np.load(datasets_path + "/"  + Labels_name + ".npy")

(53785, 749, 3)
(53785, 749, 1)


In [ ]:
# change this directory for your machine
root_dir = './'

# define the number of folds
n_folds = 5

# perform cross-validation for each dataset and algorithm combination
for dataset in datasets:
    Dataset_name = dataset + "_Dataset"
    Dataset = np.load(datasets_path + "/" + Dataset_name + ".npy")
    start = time.time() ##Start timing
    print(f"Starting to work on {Dataset_name} at {start}")

    Labels_name = dataset + "_Labels"
    Labels = np.load(datasets_path + "/"  + Labels_name + ".npy")
    


    # Create a folder for results
    results_path = root_dir + "Results/" + Dataset_name
    if os.path.exists(results_path):
        pass
    else:
        try:
            os.makedirs(results_path)
        except:
            # in case another machine created the path meanwhile !:(
            pass

        
    t_total = time.time() ##Start timing
    


    print(f"\n The dataset shape is:{Dataset.shape}")
    print(f"\n The number of data samples (N) is:{Dataset.shape[0]}")
    print(f"\n The number of TS length (T) is:{Dataset.shape[1]}")
    print(f"\n The number of TS dimention (M) is:{Dataset.shape[2]}")

    nb_classes = len(np.unique(Labels, axis=0))
    
    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
    Labels = enc.fit_transform(Labels.reshape(-1, 1)).toarray()


    kf = KFold(n_splits=n_folds, shuffle=True)
    accuracy_scores = []
    f1_scores = []
    confusion_matrices = []
    report_list = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(Dataset)):
        # split the data into training and testing sets
        X_train, X_test = Dataset[train_idx], Dataset[test_idx]
        y_train, y_test = Labels[train_idx], Labels[test_idx]
        
        # save orignal y because later we will use binary
        y_true = np.argmax(y_test, axis=1)
        
        input_shape = X_train.shape[1:]
        
        ## Create Classification module
        from dl4tsc.classifiers import mlp
        classifier = mlp.Classifier_MLP(output_directory=results_path, input_shape=input_shape,
                                        nb_classes=nb_classes, verbose=True)
        
        # fit the algorithm on the training data
        accuracy, f1, confusion, report = classifier.fit(X_train, y_train, X_test, y_test, y_true)
            
        # calculate the evaluation metrics
        accuracy_scores.append(accuracy)
        print(accuracy)

        f1_scores.append(f1)
        print(f1)

        confusion_matrices.append(confusion)
        print(confusion)

        report_list.append(report)
        print(report)
        
        print(f" fold {fold+1} is Finished!")
        keras.backend.clear_session()
        
        # save the output to a text file
        with open(f'{results_path}/dataset_{Dataset_name}_MLP_fold_{fold+1}.txt', 'w') as f:
            f.write(f'Accuracy: {accuracy}\n')
            f.write(f'F1 Score: {f1}\n')
            f.write(f'Confusion Matrix:\n{confusion}\n\n')
            f.write(f'Classification report:\n{report}\n\n')
        
    with open(f'{results_path}/dataset_{Dataset_name}_MLP.txt', 'w') as f:
        f.write("Mean accuracy: {:.4f} (std={:.3f})\n".format(np.mean(accuracy_scores), np.std(accuracy_scores)))
        f.write("Mean F1 score: {:.4f} (std={:.3f})\n".format(np.mean(f1_scores), np.std(f1_scores)))
        f.write("Mean confusion matrix:\n{}\n".format(np.array2string(np.mean(confusion_matrices, axis=0))))
        f.write("Total time elapsed: {:.4f}s".format(time.time() - t_total))

    print(" Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    print(f"Working on {Dataset_name} finished successfully!")

    



 The dataset shape is:(53785, 749, 3)

 The number of data samples (N) is:53785

 The number of TS length (T) is:749

 The number of TS dimention (M) is:3
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 749, 3)]          0         
                                                                 
 flatten_2 (Flatten)         (None, 2247)              0         
                                                                 
 dropout_8 (Dropout)         (None, 2247)              0         
                                                                 
 dense_8 (Dense)             (None, 500)               1124000   
                                                                 
 dropout_9 (Dropout)         (None, 500)               0         
                                                                 
 dense_9 (Dense)             (None,

2690/2690 [==============================] - 7s 3ms/step - loss: 0.2858 - accuracy: 0.8930 - val_loss: 0.1808 - val_accuracy: 0.9385 - lr: 0.0010
Epoch 92/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2800 - accuracy: 0.8953 - val_loss: 0.1810 - val_accuracy: 0.9389 - lr: 0.0010
Epoch 93/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2800 - accuracy: 0.8949 - val_loss: 0.1787 - val_accuracy: 0.9394 - lr: 0.0010
Epoch 94/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2755 - accuracy: 0.8988 - val_loss: 0.1773 - val_accuracy: 0.9396 - lr: 0.0010
Epoch 95/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2766 - accuracy: 0.8966 - val_loss: 0.1754 - val_accuracy: 0.9405 - lr: 0.0010
Epoch 96/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2701 - accuracy: 0.9009 - val_loss: 0.1769 - val_accuracy: 0.9377 - lr: 0.0010
Epoch 97/5000
2690/2690 [=============================

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import pandas as pd

def calculate_metrics(y_true, y_pred, duration, y_true_val=None, y_pred_val=None):
    res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall', 'duration'])
    res['precision'] = precision_score(y_true, y_pred, average='macro')
    res['accuracy'] = accuracy_score(y_true, y_pred)

    if not y_true_val is None:
        # this is useful when transfer learning is used with cross validation
        res['accuracy_val'] = accuracy_score(y_true_val, y_pred_val)

    res['recall'] = recall_score(y_true, y_pred, average='macro')
    res['duration'] = duration
    return res

In [2]:
df_metrics = calculate_metrics(y_true, y_pred, 0.0)

NameError: name 'y_true' is not defined

In [ ]:
classifier.predict(X_test, y_true,X_train,y_train,y_test,return_df_metrics = True)

In [2]:
import h5py
f = h5py.File('./Results/PHM2022_Multivar_Datasetbest_model.hdf5', 'r')

In [4]:
list(f.keys())

['model_weights', 'optimizer_weights']